In [1]:
import pandas as pd

In [64]:
import numpy as np

Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [128]:
ratings = pd.read_csv('ratings.csv', sep=',')
ratings_gr_ct = ratings.groupby('userId').agg({'rating': 'count'}).reset_index()
ratings_gr_ct = ratings_gr_ct[ratings_gr_ct.rating>100]
#ratings_gr_ct.head(10)
ratings_gr_us = ratings.groupby('userId').agg({'timestamp': ['min', 'max']}).reset_index()

ratings_gr_us['def'] = ratings_gr_us.timestamp['max'] - ratings_gr_us.timestamp['min']

ratings_gr_us = ratings_gr_us.groupby('userId').agg({'def': 'sum'}).reset_index()

#ratings_gr_us = ratings_gr_us[['userId','def']].reset_index()

#ratings_gr_ct.head(10)
#ratings_gr_us.head()


rat_m = ratings_gr_ct.merge(ratings_gr_us, on = 'userId')

#len(rat_m) == len(ratings_gr_ct)

#rat_m.head()
print(round(rat_m.agg({'def':'mean'}))/60/60/24,' days')

def    517.724167
dtype: float64  days


e:\Users\chefranov_as\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [116]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [117]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,auto_revenue,client_id
0,57483,113
1,83,114
2,912,115
3,4834,116
4,98,117


In [118]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,air_revenue,client_id
0,81,115
1,4,116
2,13,117
3,173,118


In [119]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,address,client_id
0,Комсомольская 4,111
1,Энтузиастов 8а,112
2,Левобережная 1а,113
3,Мира 14,114
4,ЗЖБИиДК 1,115
5,Строителей 18,116
6,Панфиловская 33,117
7,Мастеркова 4,118


In [124]:
all_rev = rzd.merge(auto, on = 'client_id', how='outer').merge(air, on = 'client_id', how='outer')

all_rev

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [126]:
all_rev_adr = all_rev.merge(client_base,  on = 'client_id', how='outer')
all_rev_adr

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.